In [1]:
import os
%pwd


'd:\\Python\\Practice\\Deep_Learning\\Project\\Chicken_Diesese_Classification\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Python\\Practice\\Deep_Learning\\Project\\Chicken_Diesese_Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelevalConfg:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    

In [8]:
from src.ChickenClassf.constants import *
from src.ChickenClassf.utils.common import read_yaml, create_directories,save_json


In [16]:
class ConfigurationManager:
    def __init__(self,
                config_path = CONFIG_FILE_PATH,
                params_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifact_roots])
        
    def get_validation_config(self)-> ModelevalConfg:
        train_path = self.config.training
        eval_config = ModelevalConfg(
            path_of_model = train_path.trained_model_path,
            training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images"),
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
    
        return eval_config
    

In [9]:
from urllib.parse import urlparse

In [11]:
import tensorflow as tf
model = tf.keras.models.load_model('artifacts/training/model.keras')

In [14]:

class Evaluation:
    def __init__(self, config: ModelevalConfg):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [17]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-03-01 23:54:52,881: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-01 23:54:52,887: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-01 23:54:52,888: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


c:\Users\vaibh\anaconda3\envs\DeepLearning\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.7985 - loss: nan
[2024-03-01 23:58:57,077: INFO: common: json file saved at: scores.json]
